In [1]:
import time

import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Softmax
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Accuracy, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build
from HW_keras import hw_layer

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
x_min = np.min(x_train, axis=(1, 2), keepdims=True)
x_max = np.max(x_train, axis=(1, 2), keepdims=True)
x_train = (x_train - x_min) / (x_max - x_min)

In [5]:
x_min = np.min(x_test, axis=(1, 2), keepdims=True)
x_max = np.max(x_test, axis=(1, 2), keepdims=True)
x_test = (x_test - x_min) / (x_max - x_min)

In [6]:
evaluate_list = [evaluate_build(x_test[..., i], 8) for i in range(x_test.shape[-1])]
evaluate_focus_list = [focus_build(evaluate, 0.8) for evaluate in evaluate_list]

evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 1999.43it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 2665.80it/s]


In [7]:
x = Input(shape=((32, 32, 3)))
y = hw_layer(evaluate_focus_list)(x)
y = Conv2D(10, 32, 32)(y)
y = Flatten()(y)
y = Activation('softmax')(y)

model = Model(x, y)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
hw_layer (hw_layer)          (None, 32, 32, 24)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 1, 10)          245770    
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 245,770
Trainable params: 245,770
Non-trainable params: 0
_________________________________________________________________


In [12]:
x = Input(shape=((32, 32, 3)))
y = hw_layer(evaluate_focus_list)(x)
y = Conv2D(64, 3, 1, 'same')(y)
y = Conv2D(64, 1, 1)(y)
y = MaxPooling2D(2)(y) * 8
y = Softmax(axis=-1)(y)
y = Conv2D(64, 3, 1, 'same')(y)
y = Conv2D(64, 1, 1)(y)
y = MaxPooling2D(2)(y) * 8
y = Softmax(axis=-1)(y)
y = Conv2D(64, 3, 1, 'same')(y)
y = Conv2D(64, 1, 1)(y)
y = MaxPooling2D(2)(y) * 8
y = Softmax(axis=-1)(y)
y = Flatten()(y)
y = Dense(64)(y)
y = Dense(10)(y)
y = Activation('softmax')(y)

model = Model(x, y)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
hw_layer_2 (hw_layer)        (None, 32, 32, 24)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        13888     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        4160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
tf.math.multiply_3 (TFOpLamb (None, 16, 16, 64)        0         
_________________________________________________________________
softmax_3 (Softmax)          (None, 16, 16, 64)        0   

In [15]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['AUC', 'accuracy'])

In [10]:
datagen = ImageDataGenerator(
    featurewise_center = False,  # 将整个数据集的均值设为 0
    samplewise_center = False,  # 将每个样本的均值设为 0
    featurewise_std_normalization = False,  # 将输入除以整个数据集的标准差
    samplewise_std_normalization = False,  # 将输入除以其标准差
    zca_whitening = False,  # 运用 ZCA 白化
    zca_epsilon = 1e-06,  # ZCA 白化的 epsilon 值
    rotation_range = 0,  # 随机旋转图像范围 (角度, 0 to 180)
    width_shift_range = 0.1,  # 随机水平移动图像 (总宽度的百分比)
    height_shift_range = 0.1,  # 随机垂直移动图像 (总高度的百分比)
    shear_range = 0.,  # 设置随机裁剪范围
    zoom_range = 0.,  # 设置随机放大范围
    channel_shift_range = 0.,  # 设置随机通道切换的范围
    fill_mode = 'nearest',  # 设置填充输入边界之外的点的模式
    cval = 0.,  # 在 fill_mode = "constant" 时使用的值
    horizontal_flip = True,  # 随机水平翻转图像
    vertical_flip = True,  # 随机垂直翻转图像
    rescale = None,  # 设置缩放因子 (在其他转换之前使用)
    preprocessing_function = None,  # 设置将应用于每一个输入的函数
    data_format = None,  # 图像数据格式，"channels_first" 或 "channels_last" 之一
    validation_split=0.0)  # 保留用于验证的图像比例（严格在 0 和 1 之间）

# 计算特征标准化所需的计算量
# (如果应用 ZCA 白化，则为 std，mean和主成分).
datagen.fit(x_train)

In [ ]:
model_name = time.strftime('CIFAR10_KERAS_HWNET%Y%m%d%H%M%S')

callback_list = [
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

# model.fit(x_train, y_train, epochs=100, batch_size=500, validation_data=(x_test, y_test), callbacks=callback_list)
model.fit(datagen.flow(x_train, y_train, batch_size=500), epochs=100, validation_data=(x_test, y_test), callbacks=callback_list)

Epoch 1/100
100/100 [==============================] - 36s 340ms/step - loss: 2.0791 - auc: 0.7004 - accuracy: 0.2294 - val_loss: 1.8447 - val_auc: 0.7893 - val_accuracy: 0.3168
Epoch 2/100
 44/100 [============>.................] - ETA: 16s - loss: 1.8212 - auc: 0.7951 - accuracy: 0.3199